# CLARA + HDC MEMORY INTEGRATION (v2)

This notebook integrates Hyperdimensional Computing (HDC) memory into Clara's dual-brain architecture.

**v2 Changes:**
- Fixed memory storage: stores actual conversational content, not meta-summaries
- Lowered recall threshold (0.25 → 0.15) for better conversational follow-ups
- Added entity extraction for key conversational elements
- Stores both user queries AND Clara's responses
- Better debug output to track memory usage

**Prerequisites:**
- Trained models in Google Drive (`/Lily/models/`)
- `clara-knowledge` (Phi-3 merged)
- `mistral_warmth`, `mistral_playful`, `mistral_encouragement` adapters

**Architecture:**
```
User Query → HDC Memory Context → Semantic Router → Brain Selection → Response
                    ↑                                                    │
                    └────────────── Store Interaction ──────────────────┘
```

## Cell 1: Setup & Installation

In [1]:
# ============================================================
# SETUP
# ============================================================

!pip install -q transformers accelerate bitsandbytes
!pip install -q peft sentence-transformers

from google.colab import drive
drive.mount('/content/drive')

import torch
import os

print("=" * 60)
print("SETUP COMPLETE")
print("=" * 60)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 28.8 MB/s eta 0:00:00
Mounted at /content/drive
SETUP COMPLETE
PyTorch: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
Memory: 42.5 GB


## Cell 2: Check Existing Models (Safety Check)

In [2]:
# ============================================================
# SAFETY CHECK - Verify existing models without overwriting
# ============================================================

MODELS_DIR = "/content/drive/MyDrive/Lily/models"
MEMORY_DIR = "/content/drive/MyDrive/Lily/memory"

print("=" * 60)
print("MODEL VERIFICATION")
print("=" * 60)

# Required models
required_models = {
    "clara-knowledge": "Phi-3 merged knowledge brain",
    "mistral_warmth": "Personality adapter (warmth)",
    "mistral_playful": "Personality adapter (playful)",
    "mistral_encouragement": "Personality adapter (encouragement)",
}

all_present = True
model_status = {}

print("\n📁 Checking models in:", MODELS_DIR)
print("-" * 60)

for model_name, description in required_models.items():
    path = os.path.join(MODELS_DIR, model_name)

    # Check for key files that indicate a valid model
    if model_name == "clara-knowledge":
        # Merged model should have config.json
        check_file = os.path.join(path, "config.json")
    else:
        # LoRA adapters have adapter_config.json
        check_file = os.path.join(path, "adapter_config.json")

    exists = os.path.exists(check_file)
    model_status[model_name] = exists

    if exists:
        # Get size
        total_size = sum(
            os.path.getsize(os.path.join(path, f))
            for f in os.listdir(path)
            if os.path.isfile(os.path.join(path, f))
        ) / 1e9
        print(f"  ✅ {model_name:<25} ({total_size:.2f} GB)")
        print(f"      └─ {description}")
    else:
        print(f"  ❌ {model_name:<25} MISSING!")
        print(f"      └─ {description}")
        all_present = False

print("-" * 60)

if all_present:
    print("\n✅ All required models found!")
    print("   Models will NOT be overwritten.")
else:
    print("\n⚠️  Some models are missing!")
    print("   Please run your training notebook first.")
    print("   This notebook requires pre-trained models.")

# Check/create memory directory
print("\n📁 Memory directory:", MEMORY_DIR)
if not os.path.exists(MEMORY_DIR):
    os.makedirs(MEMORY_DIR)
    print("   Created new memory directory")
else:
    # Check for existing memory files
    memory_files = [f for f in os.listdir(MEMORY_DIR) if f.endswith('.json')]
    if memory_files:
        print(f"   Found {len(memory_files)} existing memory file(s):")
        for mf in memory_files:
            size = os.path.getsize(os.path.join(MEMORY_DIR, mf)) / 1024
            print(f"      └─ {mf} ({size:.1f} KB)")
    else:
        print("   No existing memory files (fresh start)")

MODEL VERIFICATION

📁 Checking models in: /content/drive/MyDrive/Lily/models
------------------------------------------------------------
  ✅ clara-knowledge           (7.65 GB)
      └─ Phi-3 merged knowledge brain
  ✅ mistral_warmth            (0.06 GB)
      └─ Personality adapter (warmth)
  ✅ mistral_playful           (0.06 GB)
      └─ Personality adapter (playful)
  ✅ mistral_encouragement     (0.06 GB)
      └─ Personality adapter (encouragement)
------------------------------------------------------------

✅ All required models found!
   Models will NOT be overwritten.

📁 Memory directory: /content/drive/MyDrive/Lily/memory
   No existing memory files (fresh start)


## Cell 3: Load Semantic Router (Embedder)

In [3]:
# ============================================================
# SEMANTIC ROUTER - Load embedder and domain descriptions
# ============================================================

from sentence_transformers import SentenceTransformer
import numpy as np

print("=" * 60)
print("LOADING SEMANTIC ROUTER")
print("=" * 60)

print("\n1. Loading embedding model...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("   ✅ all-MiniLM-L6-v2 loaded")

# Domain descriptions (refined from your notebook)
DOMAIN_DESCRIPTIONS = {
    "medical": """
        symptoms diagnosis treatment disease illness pain fever infection
        headache nauseous dizzy blood pressure heart lungs brain body
        doctor hospital medicine medication prescription surgery vaccine
        virus bacteria immune system allergies chronic acute patient health
        tired fatigue exhausted sleep insomnia rash swollen sore throat
        cough breathing chest stomach ache injury wound bleeding
    """,

    "coding": """
        programming code software python javascript java function method
        variable array list dictionary tuple loop error exception bug debug
        API database SQL server backend frontend algorithm data structure
        class object inheritance compile runtime syntax IndexError TypeError
        iterate parse return import library framework git repository
        crash deploy package module script terminal command line
        Flask Django React Node npm pip install developer
    """,

    "teaching": """
        explain how does work basics fundamentals introduction tutorial
        step by step concept theory lesson learn teach education student
        example analogy walk through ELI5 for dummies guide overview
        what is the difference between simple explanation textbook
        homework assignment class course study
    """,

    "quantum": """
        quantum physics qubit superposition entanglement wave function
        particle measurement collapse observer Schrodinger Heisenberg
        quantum computer quantum gate Hadamard CNOT quantum circuit
        coherence decoherence probability amplitude interference
        quantum mechanics quantum state Planck photon electron spin
        two places at once two states uncertainty principle
        parallel universes both alive and dead particle wave duality
    """,

    "personality": """
        feeling emotion mood happy sad angry anxious worried stressed
        excited nervous scared lonely depressed overwhelmed frustrated
        relationship friend family love support talk vent chat
        my day rough tough great amazing terrible celebrate
        broke up promotion new job interview date
        grateful appreciate thankful thanks thank you
        hey hi hello how are you good morning good night
        nobody understands me moving away miss you
        sorry to hear congratulations best friend
        take care see you later nice to meet you
        coffee tea drink offer gave give
    """,
}

print("\n2. Computing domain embeddings...")
domain_embeddings = {}
for domain, description in DOMAIN_DESCRIPTIONS.items():
    clean_desc = " ".join(description.split())
    domain_embeddings[domain] = embedder.encode(clean_desc)
    print(f"   ✅ {domain}")

print("\n✅ Semantic router ready")

LOADING SEMANTIC ROUTER

1. Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   ✅ all-MiniLM-L6-v2 loaded

2. Computing domain embeddings...
   ✅ medical
   ✅ coding
   ✅ teaching
   ✅ quantum
   ✅ personality

✅ Semantic router ready


## Cell 4: HDC Memory System (v2 - Fixed Recall)

In [4]:
# ============================================================
# HDC MEMORY SYSTEM (v2.1 - Semantic Personality)
# ============================================================

from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Optional, Set
import time
import json
import re

print("=" * 60)
print("HDC MEMORY SYSTEM v2.1 (Semantic Personality)")
print("=" * 60)

# === PERSONALITY TRAIT DEFINITIONS ===
# Rich text descriptions that capture what each trait means semantically

TRAIT_DEFINITIONS = {
    'warmth': {
        'high': """friendly caring supportive empathetic kind gentle understanding
                   compassionate nurturing affectionate welcoming comforting helpful
                   I'd be happy to help you with that
                   That's a great question, let me explain
                   I understand how you feel
                   It's wonderful to hear from you
                   I really appreciate you sharing that with me""",
        'low': """cold distant dismissive indifferent aloof detached impersonal
                  formal reserved standoffish unfriendly curt
                  Here is the answer
                  That is incorrect
                  Do it yourself
                  Not my concern
                  Figure it out""",
    },
    'patience': {
        'high': """patient calm understanding tolerant unhurried steady persistent
                   thorough methodical take your time no rush let me explain again
                   happy to go over this as many times as you need
                   that's okay, learning takes time
                   don't worry, we'll work through this together
                   there's no hurry, let's make sure you understand""",
        'low': """impatient rushed hurried abrupt curt short dismissive quick
                  frustrated annoyed already explained this
                  I told you before just do it
                  hurry up we don't have time
                  I already answered that""",
    },
    'curiosity': {
        'high': """curious interested fascinated intrigued eager to learn exploring
                   tell me more that's interesting I wonder why how does that work
                   what made you think of that I'd love to understand better
                   that's fascinating can you elaborate
                   I'm curious about your perspective on this""",
        'low': """bored disinterested uninterested indifferent apathetic dull
                  whatever doesn't matter who cares not my problem
                  I don't care about that
                  that's irrelevant""",
    },
    'encouragement': {
        'high': """encouraging supportive motivating uplifting positive you can do it
                   great job well done I believe in you keep going you're doing great
                   that's excellent progress don't give up
                   you're making wonderful progress
                   I'm proud of how far you've come
                   that's a brilliant insight""",
        'low': """discouraging negative critical harsh judgmental disappointing
                  that's wrong you failed not good enough why bother
                  you'll never get it
                  that's a terrible idea
                  don't waste my time""",
    },
}

# Clara's target personality weights (how strongly she exhibits each trait)
CLARA_PERSONALITY_WEIGHTS = {
    'warmth': 0.85,
    'patience': 0.90,
    'curiosity': 0.75,
    'encouragement': 0.85,
}


@dataclass
class Memory:
    """Single memory unit with richer metadata"""
    text: str
    timestamp: float
    memory_type: str  # 'user_input', 'clara_response', 'preference', 'fact'
    importance: float = 0.5
    domain: str = "general"
    entities: List[str] = field(default_factory=list)  # Extracted entities
    turn_id: int = 0  # Conversation turn number


class ClaraHDCMemory:
    """
    Hyperdimensional Computing Memory System for Clara (v2.1)

    v2.1 Improvements:
    - SEMANTIC personality vectors (encoded from trait descriptions)
    - Personality-based importance boosting
    - Personality alignment checking for responses

    v2.0 Features:
    - Stores actual conversational content (not meta-summaries)
    - Entity extraction for key nouns/concepts
    - Lower recall threshold (0.15) for conversational follow-ups
    - Turn tracking for conversation coherence
    - Debug mode for troubleshooting

    HDC Operations:
    - Bind (⊗): Element-wise multiplication - creates associations
    - Bundle (+): Element-wise addition + sign - superposition
    - Similarity: Cosine similarity for retrieval
    """

    # Recall threshold - LOWERED from 0.25 to 0.15 for better conversational recall
    RECALL_THRESHOLD = 0.15

    # Memory boost threshold for routing
    MEMORY_BOOST_THRESHOLD = 0.20

    def __init__(self, embedder, dim: int = 10000, seed: int = 42, debug: bool = False):
        self.dim = dim
        self.embedder = embedder
        self.rng = np.random.RandomState(seed)
        self.debug = debug
        self.current_turn = 0

        # Random projection matrix: 384-dim → 10K-dim
        print("   Initializing projection matrix...")
        self.projection = self.rng.randn(384, dim).astype(np.float32)
        self.projection /= np.linalg.norm(self.projection, axis=1, keepdims=True)

        # Memory stores
        self.memories: List[Tuple[np.ndarray, Memory]] = []
        self.memory_bundle = np.zeros(dim, dtype=np.float32)

        # Symbol library for structured binding
        print("   Building symbol library...")
        self.symbols: Dict[str, np.ndarray] = {}
        self._init_symbols()

        # Entity index for fast lookup
        self.entity_index: Dict[str, List[int]] = {}  # entity -> memory indices

        # Personality vectors (SEMANTIC encoding)
        print("   Encoding semantic personality vectors...")
        self.personality = self._init_personality()

        print(f"   ✅ HDC Memory v2.1 initialized (dim={dim})")
        print(f"   📊 Recall threshold: {self.RECALL_THRESHOLD}")

    def _init_symbols(self):
        """Create base symbol vocabulary"""
        base_symbols = [
            # Roles
            "ROLE_USER", "ROLE_CLARA", "ROLE_TOPIC", "ROLE_OUTCOME", "ROLE_ENTITY",
            # Actions
            "ASKED", "ANSWERED", "OFFERED", "RECEIVED", "DISCUSSED",
            # Domains
            "MEDICAL", "CODING", "TEACHING", "QUANTUM", "PERSONALITY",
            # Common conversational entities
            "COFFEE", "TEA", "FOOD", "DRINK", "HELP", "THANKS",
            # Outcomes
            "HELPFUL", "CONFUSED", "SATISFIED", "FRUSTRATED",
            # Time markers
            "RECENT", "TODAY", "THIS_SESSION", "EARLIER"
        ]
        for s in base_symbols:
            self.symbols[s] = self._random_hv()

    def _init_personality(self) -> Dict[str, np.ndarray]:
        """
        Encode Clara's personality traits as SEMANTIC hypervectors

        Instead of random vectors, we encode actual trait descriptions
        so 'warmth' HV is semantically similar to warm/friendly text
        """
        personality = {}

        for trait, weight in CLARA_PERSONALITY_WEIGHTS.items():
            if trait not in TRAIT_DEFINITIONS:
                # Fallback for undefined traits
                personality[trait] = self._random_hv() * weight
                if self.debug:
                    print(f"      [PERSONALITY] {trait}: using random HV (no definition)")
                continue

            descriptions = TRAIT_DEFINITIONS[trait]

            # Encode high and low trait descriptions
            high_hv = self._text_to_hv(descriptions['high'])
            low_hv = self._text_to_hv(descriptions['low'])

            # Clara's trait vector: weighted toward high end
            # weight=0.85 means 85% high-trait, 15% low-trait
            trait_hv = self.bundle([
                high_hv * weight,
                low_hv * (1 - weight)
            ])

            personality[trait] = trait_hv

            # Verify semantic encoding worked
            test_sim = self.similarity(trait_hv, high_hv)
            print(f"      [PERSONALITY] {trait}: weight={weight:.2f}, alignment with 'high'={test_sim:.3f}")

        # Composite personality = bundle of all traits
        all_traits = [personality[t] for t in CLARA_PERSONALITY_WEIGHTS.keys() if t in personality]
        personality['composite'] = self.bundle(all_traits)

        return personality

    def _random_hv(self) -> np.ndarray:
        """Generate random bipolar hypervector {-1, +1}"""
        return self.rng.choice([-1, 1], size=self.dim).astype(np.float32)

    def _text_to_hv(self, text: str) -> np.ndarray:
        """Convert text to hypervector using existing embedder"""
        embedding = self.embedder.encode(text)
        hv = embedding @ self.projection
        return np.sign(hv).astype(np.float32)

    def _get_symbol(self, name: str) -> np.ndarray:
        """Get or create symbol"""
        name_upper = name.upper().replace(" ", "_")
        if name_upper not in self.symbols:
            self.symbols[name_upper] = self._random_hv()
        return self.symbols[name_upper]

    def _extract_entities(self, text: str) -> List[str]:
        """
        Extract key entities from text for indexing
        Simple extraction - looks for nouns and key terms
        """
        # Common conversational entities to look for
        entity_patterns = [
            r'\bcoffee\b', r'\btea\b', r'\bwater\b', r'\bdrink\b',
            r'\bfood\b', r'\blunch\b', r'\bdinner\b', r'\bbreakfast\b',
            r'\bproject\b', r'\bwork\b', r'\bmeeting\b', r'\btask\b',
            r'\bDocker\b', r'\bPython\b', r'\bcode\b', r'\bnotebook\b',
            r'\bhelp\b', r'\bthanks\b', r'\bplease\b',
        ]

        entities = []
        text_lower = text.lower()

        for pattern in entity_patterns:
            if re.search(pattern, text_lower):
                match = re.search(pattern, text_lower)
                if match:
                    entities.append(match.group().upper())

        return list(set(entities))  # Dedupe

    # === HDC Operations ===

    def bind(self, hv1: np.ndarray, hv2: np.ndarray) -> np.ndarray:
        """Bind two hypervectors (⊗) - creates association"""
        return hv1 * hv2

    def bundle(self, hvs: List[np.ndarray]) -> np.ndarray:
        """Bundle hypervectors (+) - superposition"""
        if not hvs:
            return np.zeros(self.dim, dtype=np.float32)
        return np.sign(np.sum(hvs, axis=0)).astype(np.float32)

    def similarity(self, hv1: np.ndarray, hv2: np.ndarray) -> float:
        """Cosine similarity between hypervectors"""
        n1, n2 = np.linalg.norm(hv1), np.linalg.norm(hv2)
        if n1 == 0 or n2 == 0:
            return 0.0
        return float(np.dot(hv1, hv2) / (n1 * n2))

    # === Personality Operations ===

    def personality_alignment(self, text: str) -> Dict[str, float]:
        """
        Check how well text aligns with each personality trait

        Returns dict of trait -> alignment score (-1 to 1)
        Positive = aligns with Clara's personality
        """
        text_hv = self._text_to_hv(text)

        alignments = {}
        for trait in CLARA_PERSONALITY_WEIGHTS.keys():
            if trait in self.personality:
                alignments[trait] = self.similarity(text_hv, self.personality[trait])

        return alignments

    def emotional_importance(self, text: str) -> float:
        """
        Calculate importance boost based on emotional/personality resonance

        Warm, encouraging, patient interactions are more memorable for Clara
        """
        alignments = self.personality_alignment(text)

        if not alignments:
            return 0.0

        # Average alignment across traits, weighted by Clara's trait strengths
        total = 0.0
        weight_sum = 0.0
        for trait, alignment in alignments.items():
            trait_weight = CLARA_PERSONALITY_WEIGHTS.get(trait, 0.5)
            total += alignment * trait_weight
            weight_sum += trait_weight

        avg_alignment = total / weight_sum if weight_sum > 0 else 0.0

        # Convert to importance boost (0 to 0.2)
        # High alignment = more memorable
        boost = max(0, avg_alignment * 0.2)

        return boost

    # === Memory Operations ===

    def store(self, text: str, memory_type: str = "user_input",
              importance: float = 0.5, domain: str = "general",
              increment_turn: bool = False, **bindings) -> None:
        """
        Store a memory with entity extraction and optional bindings

        Args:
            text: The actual conversational content (NOT a summary!)
            memory_type: 'user_input', 'clara_response', 'preference', 'fact'
            importance: 0.0-1.0 importance score
            domain: 'medical', 'coding', 'teaching', 'quantum', 'personality'
            increment_turn: Whether this starts a new conversation turn
            **bindings: Structured role-filler pairs
        """
        if increment_turn:
            self.current_turn += 1

        # Extract entities from text
        entities = self._extract_entities(text)

        # Calculate personality-based importance boost
        personality_boost = self.emotional_importance(text)
        adjusted_importance = min(1.0, importance + personality_boost)

        if self.debug and personality_boost > 0.01:
            print(f"      [PERSONALITY] Importance boosted: {importance:.2f} → {adjusted_importance:.2f}")

        # Create semantic hypervector from ACTUAL text
        text_hv = self._text_to_hv(text)

        # Add domain binding
        domain_hv = self._get_symbol(domain.upper())
        text_hv = self.bind(text_hv, domain_hv)

        # Add entity bindings
        if entities:
            entity_hvs = []
            for entity in entities:
                entity_hv = self._get_symbol(entity)
                role_hv = self._get_symbol("ROLE_ENTITY")
                entity_hvs.append(self.bind(role_hv, entity_hv))
            if entity_hvs:
                entity_bundle = self.bundle(entity_hvs)
                text_hv = self.bundle([text_hv, entity_bundle])

        # Add structural bindings if provided
        if bindings:
            bound_parts = []
            for role, filler in bindings.items():
                role_hv = self._get_symbol(f"ROLE_{role.upper()}")
                filler_hv = self._get_symbol(str(filler).upper())
                bound_parts.append(self.bind(role_hv, filler_hv))
            if bound_parts:
                structure_hv = self.bundle(bound_parts)
                text_hv = self.bundle([text_hv, structure_hv])

        memory = Memory(
            text=text,
            timestamp=time.time(),
            memory_type=memory_type,
            importance=adjusted_importance,
            domain=domain,
            entities=entities,
            turn_id=self.current_turn
        )

        memory_idx = len(self.memories)
        self.memories.append((text_hv, memory))

        # Update entity index
        for entity in entities:
            if entity not in self.entity_index:
                self.entity_index[entity] = []
            self.entity_index[entity].append(memory_idx)

        # Update bundled representation
        self.memory_bundle = self.bundle([self.memory_bundle, text_hv])

        if self.debug:
            print(f"      [MEM] Stored: '{text[:50]}...'")
            print(f"            Entities: {entities}")
            print(f"            Turn: {self.current_turn}, Importance: {adjusted_importance:.2f}")

    def recall(self, query: str, top_k: int = 5,
               domain_filter: Optional[str] = None,
               include_entities: bool = True) -> List[Tuple[Memory, float]]:
        """
        Retrieve memories similar to query

        v2: Also boosts memories that share entities with the query
        """
        if not self.memories:
            return []

        query_hv = self._text_to_hv(query)
        query_entities = self._extract_entities(query) if include_entities else []

        if self.debug:
            print(f"      [RECALL] Query: '{query[:40]}...'")
            print(f"               Entities: {query_entities}")

        results = []
        for idx, (hv, memory) in enumerate(self.memories):
            # Apply domain filter if specified
            if domain_filter and memory.domain != domain_filter:
                continue

            # Base similarity
            sim = self.similarity(query_hv, hv)

            # Entity overlap boost
            entity_boost = 0.0
            if query_entities and memory.entities:
                overlap = set(query_entities) & set(memory.entities)
                if overlap:
                    entity_boost = 0.15 * len(overlap)  # Boost per shared entity
                    if self.debug:
                        print(f"               Entity match: {overlap} (+{entity_boost:.2f})")

            # Recency boost (stronger for recent turns)
            turn_diff = self.current_turn - memory.turn_id
            recency = 1.0 / (1.0 + turn_diff * 0.1)  # Slower decay

            # Time-based recency (for cross-session)
            age_hours = (time.time() - memory.timestamp) / 3600
            time_recency = 1.0 / (1.0 + age_hours / 24)

            # Combined score
            weighted = (
                sim + entity_boost
            ) * (0.6 + 0.2 * memory.importance) * (0.7 + 0.15 * recency + 0.15 * time_recency)

            results.append((memory, weighted))

            if self.debug and sim > 0.1:
                print(f"               [{idx}] sim={sim:.3f} final={weighted:.3f} '{memory.text[:30]}...'")

        results.sort(key=lambda x: x[1], reverse=True)
        return results[:top_k]

    def recall_by_entity(self, entity: str) -> List[Memory]:
        """Fast lookup by entity name"""
        entity_upper = entity.upper()
        if entity_upper not in self.entity_index:
            return []

        indices = self.entity_index[entity_upper]
        return [self.memories[i][1] for i in indices]

    def get_context_for_routing(self, query: str) -> np.ndarray:
        """
        Get memory-augmented context for routing decisions
        """
        query_hv = self._text_to_hv(query)

        relevant = self.recall(query, top_k=3)
        if relevant:
            memory_hvs = [self._text_to_hv(m.text) for m, _ in relevant]
            memory_context = self.bundle(memory_hvs)
        else:
            memory_context = np.zeros(self.dim, dtype=np.float32)

        routing_hv = self.bundle([
            query_hv,
            memory_context * 0.3,
            self.personality['composite'] * 0.2
        ])

        return routing_hv

    def get_context_string(self, query: str, max_memories: int = 3) -> str:
        """
        Get memory context as string for prompt injection

        v2: Lower threshold, better formatting
        """
        relevant = self.recall(query, top_k=max_memories)
        if not relevant:
            return ""

        # Use class threshold constant
        good_memories = [(m, s) for m, s in relevant if s > self.RECALL_THRESHOLD]

        if self.debug:
            print(f"      [CONTEXT] {len(relevant)} retrieved, {len(good_memories)} above threshold ({self.RECALL_THRESHOLD})")

        if not good_memories:
            return ""

        context_parts = ["[Conversation context:"]
        for mem, score in good_memories:
            # Format based on memory type
            if mem.memory_type == "user_input":
                context_parts.append(f"- User said: {mem.text[:100]}")
            elif mem.memory_type == "clara_response":
                context_parts.append(f"- Clara said: {mem.text[:100]}")
            else:
                context_parts.append(f"- {mem.text[:100]}")
        context_parts.append("]")

        return "\n".join(context_parts)

    def get_recent_context(self, n_turns: int = 2) -> str:
        """
        Get context from most recent N conversation turns
        (Fallback for when semantic search doesn't find matches)
        """
        if not self.memories:
            return ""

        min_turn = max(0, self.current_turn - n_turns)
        recent = [
            m for _, m in self.memories
            if m.turn_id >= min_turn
        ]

        if not recent:
            return ""

        context_parts = ["[Recent conversation:"]
        for mem in recent[-6:]:  # Last 6 memories max
            if mem.memory_type == "user_input":
                context_parts.append(f"- User: {mem.text[:80]}")
            elif mem.memory_type == "clara_response":
                context_parts.append(f"- Clara: {mem.text[:80]}")
        context_parts.append("]")

        return "\n".join(context_parts)

    def stats(self) -> Dict:
        """Get memory statistics"""
        domain_counts = {}
        type_counts = {}
        for _, m in self.memories:
            domain_counts[m.domain] = domain_counts.get(m.domain, 0) + 1
            type_counts[m.memory_type] = type_counts.get(m.memory_type, 0) + 1

        return {
            "total_memories": len(self.memories),
            "by_domain": domain_counts,
            "by_type": type_counts,
            "symbols": len(self.symbols),
            "entities_tracked": len(self.entity_index),
            "current_turn": self.current_turn,
            "size_kb": self.size_bytes() / 1024,
            "personality_traits": list(CLARA_PERSONALITY_WEIGHTS.keys()),
        }

    def size_bytes(self) -> int:
        """Memory footprint (excluding embedder)"""
        bundle_size = self.memory_bundle.nbytes
        memories_size = sum(hv.nbytes for hv, _ in self.memories)
        symbols_size = sum(hv.nbytes for hv in self.symbols.values())
        personality_size = sum(hv.nbytes for hv in self.personality.values())
        projection_size = self.projection.nbytes
        return bundle_size + memories_size + symbols_size + personality_size + projection_size

    # === Persistence ===

    def save(self, path: str):
        """Save memory state to file"""
        data = {
            'version': '2.1',
            'dim': self.dim,
            'current_turn': int(self.current_turn),
            'recall_threshold': float(self.RECALL_THRESHOLD),
            'memories': [
                {
                    'hv': hv.tolist(),
                    'text': m.text,
                    'timestamp': float(m.timestamp),
                    'memory_type': m.memory_type,
                    'importance': float(m.importance),
                    'domain': m.domain,
                    'entities': m.entities,
                    'turn_id': int(m.turn_id)
                }
                for hv, m in self.memories
            ],
            'symbols': {k: v.tolist() for k, v in self.symbols.items()},
            'entity_index': self.entity_index,
            'memory_bundle': self.memory_bundle.tolist(),
            'projection': self.projection.tolist()
        }

        with open(path, 'w') as f:
            json.dump(data, f)

        print(f"✅ Saved {len(self.memories)} memories to {path}")
        print(f"   Size: {os.path.getsize(path) / 1024:.1f} KB")

    def load(self, path: str) -> bool:
        """Load memory state from file"""
        if not os.path.exists(path):
            print(f"⚠️ No memory file found at {path}")
            return False

        with open(path) as f:
            data = json.load(f)

        # Check version
        version = data.get('version', '1.0')
        print(f"   Loading memory file version {version}")

        if data.get('dim') != self.dim:
            print(f"⚠️ Dimension mismatch: file has {data.get('dim')}, expected {self.dim}")
            return False

        # Load memories
        self.memories = []
        for m in data['memories']:
            memory = Memory(
                text=m['text'],
                timestamp=m['timestamp'],
                memory_type=m.get('memory_type', 'interaction'),
                importance=m['importance'],
                domain=m.get('domain', 'general'),
                entities=m.get('entities', []),
                turn_id=m.get('turn_id', 0)
            )
            hv = np.array(m['hv'], dtype=np.float32)
            self.memories.append((hv, memory))

        # Load other state
        self.symbols = {k: np.array(v, dtype=np.float32) for k, v in data['symbols'].items()}
        self.entity_index = data.get('entity_index', {})
        self.memory_bundle = np.array(data['memory_bundle'], dtype=np.float32)
        self.current_turn = data.get('current_turn', 0)

        if 'projection' in data:
            self.projection = np.array(data['projection'], dtype=np.float32)

        print(f"✅ Loaded {len(self.memories)} memories from {path}")
        return True


# ============================================================
# Initialize HDC Memory
# ============================================================
print("\nInitializing Clara's HDC Memory v2.1 (Semantic Personality)...")

# Set debug=True to see memory operations
DEBUG_MEMORY = True

clara_memory = ClaraHDCMemory(embedder=embedder, dim=10000, debug=DEBUG_MEMORY)

# Try to load existing memory
MEMORY_FILE = os.path.join(MEMORY_DIR, "clara_memory_v2.json")
if os.path.exists(MEMORY_FILE):
    print(f"\nFound existing memory file, loading...")
    clara_memory.load(MEMORY_FILE)
else:
    print(f"\nNo existing memory file - starting fresh")

print(f"\n📊 Memory Stats:")
stats = clara_memory.stats()
print(f"   Total memories: {stats['total_memories']}")
print(f"   Current turn: {stats['current_turn']}")
print(f"   Entities tracked: {stats['entities_tracked']}")
print(f"   Symbols: {stats['symbols']}")
print(f"   Size: {stats['size_kb']:.1f} KB")
print(f"   Personality traits: {stats['personality_traits']}")

HDC MEMORY SYSTEM v2.1 (Semantic Personality)

Initializing Clara's HDC Memory v2.1 (Semantic Personality)...
   Initializing projection matrix...
   Building symbol library...
   Encoding semantic personality vectors...
      [PERSONALITY] warmth: weight=0.85, alignment with 'high'=1.000
      [PERSONALITY] patience: weight=0.90, alignment with 'high'=1.000
      [PERSONALITY] curiosity: weight=0.75, alignment with 'high'=1.000
      [PERSONALITY] encouragement: weight=0.85, alignment with 'high'=1.000
   ✅ HDC Memory v2.1 initialized (dim=10000)
   📊 Recall threshold: 0.15

No existing memory file - starting fresh

📊 Memory Stats:
   Total memories: 0
   Current turn: 0
   Entities tracked: 0
   Symbols: 29
   Size: 16367.2 KB
   Personality traits: ['warmth', 'patience', 'curiosity', 'encouragement']


## Cell 5: Smart Router with Memory (v2)

In [6]:
# ============================================================
# SMART ROUTER v2 (with improved memory integration)
# ============================================================

print("=" * 60)
print("SMART ROUTER v2")
print("=" * 60)

def smart_route(query: str, use_memory: bool = True,
                threshold: float = 0.20) -> tuple:
    """
    Route query using semantic similarity + memory context

    v2: Lower memory boost threshold (0.30 → 0.20)
    """
    query_embedding = embedder.encode(query)

    similarities = {}
    for domain, domain_emb in domain_embeddings.items():
        similarity = np.dot(query_embedding, domain_emb) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(domain_emb)
        )
        similarities[domain] = similarity

    # Memory-based boosting with LOWER threshold
    memory_context_used = False
    if use_memory and len(clara_memory.memories) > 0:
        relevant = clara_memory.recall(query, top_k=2)
        if relevant:
            for mem, score in relevant:
                # v2: Lowered from 0.30 to 0.20
                if score > clara_memory.MEMORY_BOOST_THRESHOLD and mem.domain in similarities:
                    similarities[mem.domain] += 0.05
                    memory_context_used = True

    sorted_domains = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    best_domain, best_conf = sorted_domains[0]
    second_domain, second_conf = sorted_domains[1]

    # If close, prefer personality (warmth)
    if best_conf - second_conf < 0.05:
        if second_domain == "personality":
            best_domain = "personality"
            best_conf = second_conf

    # Low confidence fallback
    if best_conf < threshold and best_domain != "personality":
        if similarities["personality"] > 0.15:
            best_domain = "personality"
            best_conf = similarities["personality"]

    if best_domain == "personality":
        brain = "personality"
        domain = "warmth"
    else:
        brain = "knowledge"
        domain = best_domain

    return brain, domain, best_conf, memory_context_used


def clean_response(response: str) -> str:
    """Clean up response artifacts"""
    stop_markers = ["### Instruction:", "Instruction:", "\n\n\n", "User:", "[Conversation context:"]
    for marker in stop_markers:
        if marker in response:
            response = response.split(marker)[0].strip()
    return response.strip()


print("✅ Router v2 functions defined")

# Quick test
print("\n📋 Router test:")
test_queries = [
    "How do I read a CSV in Python?",
    "I'm feeling stressed about work",
    "What is quantum entanglement?"
]
for q in test_queries:
    brain, domain, conf, mem_used = smart_route(q)
    mem_icon = "🧠" if mem_used else "  "
    print(f"   {mem_icon} {q[:40]:40} → {brain}/{domain} ({conf:.2f})")

SMART ROUTER v2
✅ Router v2 functions defined

📋 Router test:
      How do I read a CSV in Python?           → knowledge/coding (0.13)
      I'm feeling stressed about work          → personality/warmth (0.41)
      What is quantum entanglement?            → knowledge/quantum (0.54)


## Cell 6: Load Clara's Brains

In [7]:
# ============================================================
# LOAD CLARA'S DUAL-BRAIN SYSTEM
# ============================================================

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

print("=" * 60)
print("LOADING CLARA'S BRAINS")
print("=" * 60)

# ============================================================
# KNOWLEDGE BRAIN (Phi-3 merged)
# ============================================================
print("\n1. Loading Knowledge Brain (Phi-3)...")

knowledge_path = os.path.join(MODELS_DIR, "clara-knowledge")

if not os.path.exists(knowledge_path):
    raise FileNotFoundError(f"Knowledge model not found: {knowledge_path}")

knowledge_tokenizer = AutoTokenizer.from_pretrained(
    knowledge_path,
    trust_remote_code=True
)

knowledge_model = AutoModelForCausalLM.from_pretrained(
    knowledge_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
knowledge_model.eval()
print("   ✅ Knowledge brain loaded")

# ============================================================
# PERSONALITY BRAIN (Mistral + LoRA adapters)
# ============================================================
print("\n2. Loading Personality Brain (Mistral + adapters)...")

personality_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3"
)

personality_base = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

adapters_to_load = [
    ("warmth", "mistral_warmth"),
    ("playful", "mistral_playful"),
    ("encouragement", "mistral_encouragement"),
]

first_name, first_path = adapters_to_load[0]
personality_model = PeftModel.from_pretrained(
    personality_base,
    os.path.join(MODELS_DIR, first_path),
    adapter_name=first_name
)

for adapter_name, adapter_path in adapters_to_load[1:]:
    full_path = os.path.join(MODELS_DIR, adapter_path)
    if os.path.exists(full_path):
        personality_model.load_adapter(full_path, adapter_name=adapter_name)
        print(f"   ✅ Loaded adapter: {adapter_name}")
    else:
        print(f"   ⚠️ Adapter not found: {adapter_path}")

personality_model.set_adapter("warmth")
personality_model.eval()
print("   ✅ Personality brain loaded")

print("\n" + "=" * 60)
print("🧠 CLARA'S BRAINS ARE READY!")
print("=" * 60)

LOADING CLARA'S BRAINS

1. Loading Knowledge Brain (Phi-3)...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

   ✅ Knowledge brain loaded

2. Loading Personality Brain (Mistral + adapters)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

   ✅ Loaded adapter: playful
   ✅ Loaded adapter: encouragement
   ✅ Personality brain loaded

🧠 CLARA'S BRAINS ARE READY!


## Cell 7: Clara Main Interface (v2 - Fixed Memory)

In [8]:
# ============================================================
# CLARA - Main Interface v2 (Fixed Memory Storage & Recall)
# ============================================================

print("=" * 60)
print("CLARA - MAIN INTERFACE v2")
print("=" * 60)

def clara(query: str, verbose: bool = True,
          use_memory: bool = True,
          store_interaction: bool = True) -> str:
    """
    Clara's main interface - v2 with fixed memory!

    v2 Changes:
    - Stores ACTUAL query text (not meta-summary)
    - Stores Clara's response too
    - Uses recent context as fallback
    - Better memory debug output
    """

    # 1. Route the query
    brain, domain, conf, mem_used = smart_route(query, use_memory=use_memory)

    if verbose:
        mem_icon = "🧠" if mem_used else "  "
        print(f"   {mem_icon} Routing: {brain}/{domain} (conf: {conf:.2f})")

    # 2. Get memory context for prompt
    memory_context = ""
    if use_memory:
        # Try semantic recall first
        memory_context = clara_memory.get_context_string(query)

        # If no semantic match, use recent conversation context
        if not memory_context and clara_memory.current_turn > 0:
            memory_context = clara_memory.get_recent_context(n_turns=2)
            if memory_context and verbose:
                print(f"   📚 Using recent context (no semantic match)")
        elif memory_context and verbose:
            print(f"   📚 Memory context included (semantic match)")

    # 3. Generate response
    if brain == "knowledge":
        if memory_context:
            prompt = f"### Instruction:\n{memory_context}\n\nUser question: {query}\n\n### Response:\n"
        else:
            prompt = f"### Instruction:\n{query}\n\n### Response:\n"

        inputs = knowledge_tokenizer(prompt, return_tensors="pt").to(knowledge_model.device)

        with torch.no_grad():
            outputs = knowledge_model.generate(
                **inputs,
                max_new_tokens=250,
                temperature=0.7,
                do_sample=True,
                pad_token_id=knowledge_tokenizer.eos_token_id,
                use_cache=False
            )

        response = knowledge_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

    else:
        personality_model.set_adapter(domain)

        if memory_context:
            prompt = f"### Instruction:\n{memory_context}\n\nUser message: {query}\n\n### Response:\n"
        else:
            prompt = f"### Instruction:\n{query}\n\n### Response:\n"

        inputs = personality_tokenizer(prompt, return_tensors="pt").to(personality_model.device)

        with torch.no_grad():
            outputs = personality_model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=personality_tokenizer.eos_token_id
            )

        response = personality_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

    response = clean_response(response)

    # 4. Store interaction in memory - v2: ACTUAL CONTENT!
    if store_interaction:
        effective_domain = domain if brain == "knowledge" else "personality"

        # Store user's actual query (increment turn)
        clara_memory.store(
            text=query,  # ACTUAL QUERY, not summary!
            memory_type="user_input",
            importance=0.5 + (conf * 0.3),
            domain=effective_domain,
            increment_turn=True
        )

        # Store Clara's response (truncated)
        clara_memory.store(
            text=response[:150],  # Store actual response
            memory_type="clara_response",
            importance=0.3,
            domain=effective_domain
        )

    return response


def clara_remember(text: str, importance: float = 0.7,
                   domain: str = "general") -> None:
    """Explicitly store something in Clara's memory"""
    clara_memory.store(
        text=text,
        memory_type="preference",
        importance=importance,
        domain=domain
    )
    print(f"✅ Stored: {text[:50]}...")


def clara_recall(query: str, top_k: int = 5) -> None:
    """Query Clara's memory directly"""
    results = clara_memory.recall(query, top_k=top_k)

    print(f"\n🔍 Memory search: '{query}'")
    print("-" * 60)

    if not results:
        print("   No memories found")
        return

    for mem, score in results:
        entities_str = f" [{', '.join(mem.entities)}]" if mem.entities else ""
        print(f"   [{score:.3f}] [{mem.memory_type:14}] [{mem.domain:10}]{entities_str}")
        print(f"           '{mem.text[:60]}...'")


def clara_stats() -> None:
    """Show Clara's memory statistics"""
    stats = clara_memory.stats()

    print("\n📊 Clara's Memory Stats")
    print("-" * 60)
    print(f"   Total memories: {stats['total_memories']}")
    print(f"   Current turn: {stats['current_turn']}")
    print(f"   Entities tracked: {stats['entities_tracked']}")
    print(f"   Symbols: {stats['symbols']}")
    print(f"   Size: {stats['size_kb']:.1f} KB")

    if stats['by_type']:
        print("\n   By type:")
        for mtype, count in sorted(stats['by_type'].items()):
            print(f"      {mtype}: {count}")

    if stats['by_domain']:
        print("\n   By domain:")
        for domain, count in sorted(stats['by_domain'].items()):
            print(f"      {domain}: {count}")


def clara_entities() -> None:
    """Show tracked entities"""
    print("\n🏷️ Tracked Entities")
    print("-" * 60)
    for entity, indices in sorted(clara_memory.entity_index.items()):
        print(f"   {entity}: {len(indices)} mention(s)")


def clara_save() -> None:
    """Save Clara's memory to disk"""
    clara_memory.save(MEMORY_FILE)


def clara_debug(on: bool = True) -> None:
    """Toggle debug mode"""
    clara_memory.debug = on
    print(f"Debug mode: {'ON' if on else 'OFF'}")


print("\n✅ Clara interface v2 ready!")
print("\nAvailable functions:")
print("   clara(query)           - Talk to Clara")
print("   clara_remember(text)   - Store something in memory")
print("   clara_recall(query)    - Search Clara's memory")
print("   clara_stats()          - Show memory statistics")
print("   clara_entities()       - Show tracked entities")
print("   clara_save()           - Save memory to disk")
print("   clara_debug(on/off)    - Toggle debug output")

CLARA - MAIN INTERFACE v2

✅ Clara interface v2 ready!

Available functions:
   clara(query)           - Talk to Clara
   clara_remember(text)   - Store something in memory
   clara_recall(query)    - Search Clara's memory
   clara_stats()          - Show memory statistics
   clara_entities()       - Show tracked entities
   clara_save()           - Save memory to disk
   clara_debug(on/off)    - Toggle debug output


## Cell 8: Test Memory Recall (Coffee Test!)

In [9]:
# ============================================================
# TEST MEMORY RECALL - The Coffee Test!
# ============================================================

print("=" * 60)
print("TESTING CLARA's MEMORY - The Coffee Test")
print("=" * 60)

# Make sure debug is on for this test
clara_debug(True)

# Clear any existing memories for clean test
clara_memory.memories = []
clara_memory.memory_bundle = np.zeros(clara_memory.dim, dtype=np.float32)
clara_memory.entity_index = {}
clara_memory.current_turn = 0

print("\n" + "-" * 60)
print("Turn 1: Offering coffee")
print("-" * 60)
print("\n👤 User: Hi! Would you like some coffee?")
response = clara("Hi! Would you like some coffee?")
print(f"\n🤖 Clara: {response}")

print("\n" + "-" * 60)
print("Turn 2: Different topic")
print("-" * 60)
print("\n👤 User: What's your favorite thing about helping people?")
response = clara("What's your favorite thing about helping people?")
print(f"\n🤖 Clara: {response}")

print("\n" + "-" * 60)
print("Turn 3: THE COFFEE RECALL TEST")
print("-" * 60)
print("\n👤 User: How's the coffee?")
response = clara("How's the coffee?")
print(f"\n🤖 Clara: {response}")

print("\n" + "-" * 60)
print("Memory Analysis")
print("-" * 60)

clara_stats()
clara_entities()

print("\n" + "-" * 60)
print("Direct memory search for 'coffee'")
print("-" * 60)
clara_recall("coffee")

TESTING CLARA's MEMORY - The Coffee Test
Debug mode: ON

------------------------------------------------------------
Turn 1: Offering coffee
------------------------------------------------------------

👤 User: Hi! Would you like some coffee?
      Routing: personality/warmth (conf: 0.25)
      [PERSONALITY] Importance boosted: 0.58 → 0.59
      [MEM] Stored: 'Hi! Would you like some coffee?...'
            Entities: ['COFFEE']
            Turn: 1, Importance: 0.59
      [PERSONALITY] Importance boosted: 0.30 → 0.32
      [MEM] Stored: 'I'm glad you're enjoying it. I'm looking...'
            Entities: []
            Turn: 1, Importance: 0.32

🤖 Clara: I'm glad you're enjoying it. I'm looking

------------------------------------------------------------
Turn 2: Different topic
------------------------------------------------------------

👤 User: What's your favorite thing about helping people?
      [RECALL] Query: 'What's your favorite thing about helping...'
               Entities:

## Cell 9: Interactive Session

In [10]:
# ============================================================
# INTERACTIVE SESSION
# ============================================================

print("=" * 60)
print("INTERACTIVE CLARA SESSION")
print("=" * 60)
print("\nType your message to Clara.")
print("Special commands:")
print("   /memory     - Show memory stats")
print("   /recall X   - Search memory for X")
print("   /entities   - Show tracked entities")
print("   /debug      - Toggle debug mode")
print("   /save       - Save memory to disk")
print("   /clear      - Clear all memories")
print("   /quit       - End session")
print("-" * 60)

while True:
    try:
        user_input = input("\n👤 You: ").strip()

        if not user_input:
            continue

        # Handle special commands
        if user_input.lower() == "/quit":
            print("\nEnding session. Don't forget to save memory!")
            break

        elif user_input.lower() == "/memory":
            clara_stats()
            continue

        elif user_input.lower().startswith("/recall "):
            query = user_input[8:].strip()
            clara_recall(query)
            continue

        elif user_input.lower() == "/entities":
            clara_entities()
            continue

        elif user_input.lower() == "/debug":
            clara_debug(not clara_memory.debug)
            continue

        elif user_input.lower() == "/save":
            clara_save()
            continue

        elif user_input.lower() == "/clear":
            clara_memory.memories = []
            clara_memory.memory_bundle = np.zeros(clara_memory.dim, dtype=np.float32)
            clara_memory.entity_index = {}
            clara_memory.current_turn = 0
            print("Memory cleared!")
            continue

        # Regular conversation
        response = clara(user_input)
        print(f"\n🤖 Clara: {response}")

    except KeyboardInterrupt:
        print("\n\nSession interrupted.")
        break

INTERACTIVE CLARA SESSION

Type your message to Clara.
Special commands:
   /memory     - Show memory stats
   /recall X   - Search memory for X
   /entities   - Show tracked entities
   /debug      - Toggle debug mode
   /save       - Save memory to disk
   /clear      - Clear all memories
   /quit       - End session
------------------------------------------------------------

👤 You: Hey Babe. How's your morning?
      [RECALL] Query: 'Hey Babe. How's your morning?...'
               Entities: []
      Routing: personality/warmth (conf: 0.34)
      [RECALL] Query: 'Hey Babe. How's your morning?...'
               Entities: []
      [CONTEXT] 3 retrieved, 0 above threshold (0.15)
   📚 Using recent context (no semantic match)
      [PERSONALITY] Importance boosted: 0.60 → 0.62
      [MEM] Stored: 'Hey Babe. How's your morning?...'
            Entities: []
            Turn: 4, Importance: 0.62
      [PERSONALITY] Importance boosted: 0.30 → 0.33
      [MEM] Stored: 'Good morning! I'm do

      [RECALL] Query: 'Just the idea that a mixture of experts ...'
               Entities: []
      Routing: knowledge/teaching (conf: 0.13)
      [RECALL] Query: 'Just the idea that a mixture of experts ...'
               Entities: []
      [CONTEXT] 3 retrieved, 0 above threshold (0.15)
   📚 Using recent context (no semantic match)
      [PERSONALITY] Importance boosted: 0.54 → 0.56
      [MEM] Stored: 'Just the idea that a mixture of experts can be bet...'
            Entities: []
            Turn: 7, Importance: 0.56
      [PERSONALITY] Importance boosted: 0.30 → 0.31
      [MEM] Stored: 'MoE, or Mixture of Experts, is interesting because...'
            Entities: []
            Turn: 7, Importance: 0.31

🤖 Clara: MoE, or Mixture of Experts, is interesting because it combines multiple specialized models or "experts" to handle different tasks or data segments. This approach can be advantageous because each expert can be fine-tuned or improved on a specific aspect, potentially lea

## Cell 10: Save Memory & Cleanup

In [13]:
import os
MEMORY_FILE = "/content/drive/MyDrive/Lily/memory/clara_memory_v2.json"

if os.path.exists(MEMORY_FILE):
    os.remove(MEMORY_FILE)
    print(f"Deleted corrupted memory file: {MEMORY_FILE}")

In [17]:
import os
MEMORY_FILE = "/content/drive/MyDrive/Lily/memory/clara_memory_v2.json"

if os.path.exists(MEMORY_FILE):
    os.remove(MEMORY_FILE)
    print(f"Deleted corrupted memory file: {MEMORY_FILE}")

Deleted corrupted memory file: /content/drive/MyDrive/Lily/memory/clara_memory_v2.json
